In [1]:
import numpy as np
import random

In [2]:
import networkx as nx
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\ENZYMES_g102.edges", nodetype=int)
G = nx.read_edgelist(r"..\vertex_cover_benchmarks\aves-sparrowlyon-flock-season2-unw.edges", nodetype=int)
#G = nx.read_edgelist("brock200_1_complement.edges", nodetype=int)

In [3]:
#MAX_FITNESS = G.order() # Tal y como está definido, el máximo valor que puede tomar el fitness es el número de nodos del grafo
MAX_FITNESS = 1
MAX_ITER = 1000
SAMPLE_SIZE = 100
INITIAL_SAMPLE_SIZE = 300

In [4]:
class Solution:
    def __init__(self, sol, f = 0):
        self.sol = sol
        self.f = f

    def set_f(self, f):
        self.f = f

In [5]:
def subsets(a):
  l = len(a)
  n = l - 1
  x = [0] * n
  f = [j for j in range(l)]
  r = list()
  
  while n:
    m = [0] + x # insert 0 at the beginning of the bit string forms
    subset = [a[i] for i in range(l) if m[i] == 1] # generate the subset
    
    # and use the subset to generate the subsets for when 1 is inserted at the beginning
    k = [a[0]] + subset
    
    r.append(subset) # add to the table all subsets of bits strigs with prefix 0
    r.append(k) # add to the table all subsets of bits strings with prefix 1
    
    # initialization
    # this is where we choose which j we want to change
    j = f[0]
    f[0] = 0
    
    # if j == n we terminate because we have all the possible bit strings
    if j == n:
      break
     
    # update our array when j != n
    f[j] = f[j + 1]
    f[j + 1] = j + 1
    
    x[j] = 1 - x[j] # complement coordinate j and return to line 8 if j != n
  
  return r

In [6]:
def is_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            return False
    return True

In [7]:
def naive_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            node = random.choice([u, v])
            sol[node - 1] = 1
    return

In [8]:
def fitness(sol):
    print("oye por favor")
    n = G.order()
    if is_cover(sol, G):
        return n - sum(sol)
    else:
        if sum(sol) >= (2 * n // 3):
            print("estoy en sitio bueno")
            zeros_index = []
            for i in range(len(sol)):
                if sol[i] == 0:
                    zeros_index.append(i)
                    #sol[i] = 1
                    #if is_cover(sol, G):
                    #    print(sol)
                    #    break
                    #sol[i] = 0
            r = subsets(zeros_index)
            r.sort(key=len)
            # r[0] es el conjunto vacío
            for subset in r[1:]:
                for index in subset:
                    sol[index] = 1 # Esto modifica sol porque se pasa por referencia
                if is_cover(sol, G):
                    print(f"found fitness {n - sum(sol)}")
                    return n - sum(sol)
                for index in subset:
                    sol[index] = 0
            #print("calavera 1")
            return 0
        else:
            naive_cover(sol, G)
            #print("calavera 2")
            return n - sum(sol)

In [9]:
(G.order() - 10) / G.order()

0.7619047619047619

In [9]:
def fitness2(sol):
    if is_cover(sol, G):
        print("found")
        n = G.order()
        return (n - sum(sol)) / n
    else:
        return 0

In [10]:
def fitness3(sol):
    n = G.order()
    if is_cover(sol, G):
        #print("buenísimo")
        return (n - sum(sol)) / sum(sol)
    else:
        if sum(sol) >= (2 * n // 3):
            #print("estoy en sitio bueno")
            zeros_index = []
            for i in range(len(sol)):
                if sol[i] == 0:
                    zeros_index.append(i)
                    #sol[i] = 1
                    #if is_cover(sol, G):
                    #    print(sol)
                    #    break
                    #sol[i] = 0
            r = subsets(zeros_index)
            r.sort(key=len)
            # r[0] es el conjunto vacío
            for subset in r[1:]:
                for index in subset:
                    sol[index] = 1 # Esto modifica sol porque se pasa por referencia
                if is_cover(sol, G):
                    #print(f"found fitness {(n - sum(sol)) / sum(sol)}")
                    return (n - sum(sol)) / sum(sol)
                for index in subset:
                    sol[index] = 0
            assert(False)
            naive_cover(sol, G) # Pensándolo, aquí no debería llegar porque siempre se puede completar
            print("calavera 1")
            return (n - sum(sol)) / sum(sol)
        else:
            naive_cover(sol, G)
            #print("calavera 2")
            return (n - sum(sol)) / sum(sol)

In [11]:
def fitness4(sol):
    n = G.order()
    if not is_cover(sol, G):
        naive_cover(sol, G)
    return (n - sum(sol)) / sum(sol)

In [12]:
def f(sol):
    fit = fitness4(sol.sol)
    sol.set_f(fit)
    #print(f"FITNESS de {sol.sol} = {fit}")
    return fit

In [13]:
def hamming_distance(sol1, sol2):
    n = len(sol1)
    #print(f"sol1 = {sol1}\nsol2 = {sol2}")
    assert(n == len(sol2))
    return [sol1[i] == sol2[i] for i in range(n)].count(False) ** 3

In [14]:
def inverse_distance(sol1, sol2, dist):
    d = dist(sol1, sol2)
    return 1 / d if d != 0 else 2 # Realmente la forma de manejar el infinito depende de la distancia
# El 2 funciona para Hammings porque no puede haber distancias d 0 < d < 1 (es decir, tales que 1 / d > 2).

In [16]:
inverse_distance([1,0,1,0], [0,1,0,1], hamming_distance)

0.015625

In [15]:
def w(sol1, sol2, A, dist):
    assert(len(sol1) == len(sol2))
    sum = 0
    for point in A:
        sol = point.sol
        assert(len(sol) == len(sol1))
        sum += inverse_distance(sol1, sol, dist)
    assert(sum > 0) # Creo que se puede demostrar que esto no va a ser 0.
    inv = inverse_distance(sol1, sol2, dist)
    res = inv / sum
    #print(f"inv: {inv}\nsum: {sum}\nres: {res}")
    return res

In [16]:
def g(sol, A, dist):
    sum = 0
    for point in A:
        wei = w(sol.sol, point.sol, A, dist)
        fit = point.f
        #print(f"\nwei: {wei} \n fit: {fit}")
        sum += wei * fit
    #print(f"\n\n g: {sum}\n\n")
    return sum

In [17]:
def search_optimal(A):
    card = len(A)
    assert(card > 0)
    opt = A[0]
    max_fit = opt.f
    for elem in A[1:]:
        fit = elem.f
        if fit > max_fit:
            max_fit = fit
            opt = elem
    return opt

In [18]:
def search_nearest(sol, A, dist):
    card = len(A)
    assert(card > 0)
    nearest = A[0]
    min_distance = dist(sol.sol, nearest.sol)
    for elem in A[1:]:
        distance = dist(sol.sol, elem.sol)
        if distance < min_distance:
            min_distance = distance
            nearest = elem
    return nearest

In [19]:
def sample(k, A, dist, skip = []):
    n = G.order()
    sample = []
    while len(sample) < k:
        x = [random.choice([0, 1]) for _ in range(n)]
        if x in sample or x in skip:
            continue
        #y = random.randint(0, MAX_FITNESS)
        y = random.uniform(0, MAX_FITNESS)
        sol = Solution(x)
        set_points = [search_optimal(A), search_nearest(sol, A, dist)]
        g_val = g(sol, set_points, dist)
        if g_val > y:
            #print(fitness(x))
            f(sol)
            #print(f"Muestra añadida {x}\ncon valor de g: {g_val} y valor de y: {y}")
            sample.append(sol)
    return sample

In [20]:
def init(k):
    n = G.order()
    init = []
    while len(init) < k:
        x = [random.choice([0, 1]) for _ in range(n)]
        if not any(point.sol == x for point in init): # Esto se evita usando un set. Aún no tengo claro si necesito índices, o si el set es eficiente en python.
            sol = Solution(x)
            f(sol)
            init.append(sol)
    return init

def max_k_fitness(set_sols, k):
    sor = sorted([(fitness(elem), elem) for elem in set_sols])
    return [elem for (_,elem) in sor][-k:]

def max_fitness(set_sols):
    max_value = fitness(set_sols[0])
    max_elem = set_sols[0]
    for elem in set_sols[1:]:
        f = fitness(elem)
        if f > max_value:
            max_value = f
            max_elem = elem
    return max_elem, max_value

def next_gen(curr_gen, k):
    new = sample(k, skip = curr_gen)
    return max_k_fitness(curr_gen + new, k)

def algorithm():
    gen = init(SAMPLE_SIZE)
    for _ in range(MAX_ITER):
        gen = next_gen(gen, SAMPLE_SIZE)
    return max_fitness(gen)

In [21]:
def max_f(A):
    assert(len(A) > 0)
    max_value = A[0].f
    max_elem = A[0]
    for elem in A[1:]:
        fit = elem.f
        if fit > max_value:
            max_value = fit
            max_elem = elem
    return max_elem, max_value

In [22]:
def alg(sample_size, num_iter, initial_sample_size, dist):
    #dic.clear()
    A = init(initial_sample_size)
    for i in range(num_iter):
        print(f"iter {i}")
        A.extend(sample(sample_size, A, dist, skip = A))
    return max_f(A)

In [23]:
def next_swarm_step(k, A, dist):
    opt = search_optimal(A)
    next = sample(k - 1, A, dist, skip = A)
    next.append(opt)
    return next

In [24]:
def alg2(sample_size, num_iter, dist):
    A = init(sample_size)
    for i in range(num_iter):
        print(f"iter {i}")
        A = next_swarm_step(sample_size, A, dist)
    return max_f(A)

In [27]:
"""
elem, fit = alg2(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dist = hamming_distance)
print(f'Máximo valor de fitness ({fit}) encontrado para {elem.sol}, con {sum(elem.sol)} nodos.')

print(is_cover(elem.sol, G))

num_ceros = G.order() - sum(elem.sol)
print(num_ceros, "ceros")
"""

'\nelem, fit = alg2(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dist = hamming_distance)\nprint(f\'Máximo valor de fitness ({fit}) encontrado para {elem.sol}, con {sum(elem.sol)} nodos.\')\n\nprint(is_cover(elem.sol, G))\n\nnum_ceros = G.order() - sum(elem.sol)\nprint(num_ceros, "ceros")\n'

In [28]:
#for i in range(20):
#    elem, fit = alg2(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dist = hamming_distance)
#    print(f'Máximo valor de fitness ({fit}) encontrado para {elem.sol}, con {sum(elem.sol)} nodos.')

#    print(is_cover(elem.sol, G))

#    num_ceros = G.order() - sum(elem.sol)
#    print(num_ceros, "ceros")

In [25]:
output = open(r"..\results\sparrow\medias_ponderadas_PSO.csv", "a")
for _ in range(20):
    elem, fit = alg2(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dist = hamming_distance)
    output.write(f'{elem.sol},')

    num_ceros = G.order() - sum(elem.sol)
    output.write(f'{num_ceros}\n')

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99
iter 100
iter 101
iter 102
iter 103
iter 104
iter 105
iter 106
iter 107
iter 108
iter 109
iter 110
iter 111
iter 112
iter 113
iter 114
iter 115
iter 116
iter 117
iter 118
iter 119
iter 120
iter 121
iter 122
ite

In [26]:
output.close()